In [ ]:
import os
import random
import pygame
from pygame import display, event, image, font
from time import sleep

# Constants
IMAGE_SIZE = 128
SCREEN_SIZE = 512
NUM_TILES_SIDE = 4
NUM_TILES_TOTAL = 16
MARGIN = 8

ASSET_DIR = 'assets'
ASSET_FILES = [x for x in os.listdir(ASSET_DIR) if x[-3:].lower() == 'png']
assert len(ASSET_FILES) == 8

animals_count = dict((a, 0) for a in ASSET_FILES)

def available_animals():
    return [animal for animal, count in animals_count.items() if count < 2]

class Animal:
    def __init__(self, index):
        self.index = index
        self.name = random.choice(available_animals())
        self.image_path = os.path.join(ASSET_DIR, self.name)
        self.row = index // NUM_TILES_SIDE
        self.col = index % NUM_TILES_SIDE
        self.skip = False
        self.image = image.load(self.image_path)
        self.image = pygame.transform.scale(self.image, (IMAGE_SIZE - 2 * MARGIN, IMAGE_SIZE - 2 * MARGIN))
        self.box = self.image.copy()
        self.box.fill((200, 200, 200))

        animals_count[self.name] += 1

def find_index_from_xy(x, y):
    row = y // IMAGE_SIZE
    col = x // IMAGE_SIZE
    index = row * NUM_TILES_SIDE + col
    return row, col, index

def reset_game(tiles):
    for tile in tiles:
        tile.skip = False
    return []

# Reset Button Constants
RESET_BUTTON_RECT = pygame.Rect(10, SCREEN_SIZE + 50, 100, 30)
RESET_BUTTON_COLOR_INACTIVE = (0, 0, 255)
RESET_BUTTON_COLOR_ACTIVE = (0, 0, 180)

def draw_reset_button(screen, mouse_pos, reset_button_hovered):
    color = RESET_BUTTON_COLOR_ACTIVE if reset_button_hovered else RESET_BUTTON_COLOR_INACTIVE
    pygame.draw.rect(screen, color, RESET_BUTTON_RECT)
    font.init()
    button_font = font.Font(None, 24)
    text = button_font.render("Reset", True, (255, 255, 255))
    text_rect = text.get_rect(center=RESET_BUTTON_RECT.center)
    screen.blit(text, text_rect)

def game():
    pygame.init()
    display.set_caption('Memory Matching Game')
    # Adjusted frame size to accommodate the match label and reset button
    screen = display.set_mode((SCREEN_SIZE, SCREEN_SIZE + 100))
    
    matched = image.load('other_assets/matched.png')
    running = True
    tiles = [Animal(i) for i in range(0, NUM_TILES_TOTAL)]
    current_images_displayed = []

    # Create a font for displaying the match counter
    font.init()
    match_font = font.Font(None, 36)

    # Initialize match counter
    match_counter = 0

    matched_indices = []

    # Initialize reset button state
    reset_button_hovered = False

    while running:
        current_events = event.get()

        for e in current_events:
            if e.type == pygame.QUIT:
                running = False

            # Handle mouse events
            if e.type == pygame.MOUSEBUTTONDOWN:
                mouse_x, mouse_y = pygame.mouse.get_pos()

                # Check if the mouse click is on the reset button
                if RESET_BUTTON_RECT.collidepoint(mouse_x, mouse_y):
                    current_images_displayed = reset_game(tiles)
                    match_counter = 0  # Reset the match counter
                    matched_indices = []

                # Check if the mouse click is on a tile
                row, col, index = find_index_from_xy(mouse_x, mouse_y)
                if index not in current_images_displayed and index not in matched_indices:
                    if len(current_images_displayed) > 1:
                        current_images_displayed = current_images_displayed[1:] + [index]
                    else:
                        current_images_displayed.append(index)

            # Check if the mouse hovers over the reset button
            if e.type == pygame.MOUSEMOTION:
                mouse_x, mouse_y = pygame.mouse.get_pos()
                reset_button_hovered = RESET_BUTTON_RECT.collidepoint(mouse_x, mouse_y)

            if e.type == pygame.KEYDOWN:
                if e.key == pygame.K_ESCAPE:
                    running = False

        # Display animals
        screen.fill((255, 255, 255))

        total_skipped = 0

        for i, tile in enumerate(tiles):
            current_image = tile.image if i in current_images_displayed else tile.box
            if not tile.skip:
                screen.blit(current_image, (tile.col * IMAGE_SIZE + MARGIN, tile.row * IMAGE_SIZE + MARGIN))
            else:
                total_skipped += 1

        # Check for matches
        if len(current_images_displayed) == 2:
            idx1, idx2 = current_images_displayed
            if 0 <= idx1 < len(tiles) and 0 <= idx2 < len(tiles) and tiles[idx1].name == tiles[idx2].name:
                # Display the matched image for a short duration
                tiles[idx1].skip = True
                tiles[idx2].skip = True
                sleep(0.3)
                screen.blit(matched,(0,0))
                display.flip()
                sleep(0.5)
                match_counter += 1 
                matched_indices.extend([idx1, idx2])

                current_images_displayed = []

        # Display the match label at the bottom
        match_label = match_font.render("Matches: ", True, (0, 0, 0))
        match_value = match_font.render(str(match_counter), True, (0, 0, 0))
        screen.blit(match_label, (10, SCREEN_SIZE + 10))
        screen.blit(match_value, (match_label.get_width() + 20, SCREEN_SIZE + 10))

        draw_reset_button(screen, pygame.mouse.get_pos(), reset_button_hovered)

        display.flip()

        if total_skipped == len(tiles):
            running = False

    sleep(1)

if __name__ == "__main__":
    game()
    pygame.quit()

    quit()


pygame 2.5.0 (SDL 2.28.0, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
